In [1]:
import json
import cv2
import deeplake as hub
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.datasets import CocoDetection

In [2]:
# Load the training data
img_folder = "/home/ec2-user/SageMaker/Nano-Instance-Segmentor/notebooks/Pappers-2-1/valid/"
ann_file = "/home/ec2-user/SageMaker/Nano-Instance-Segmentor/notebooks/Pappers-2-1/valid/_annotations.coco.json"

dataset = CocoDetection(img_folder, ann_file)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os

# Define the path to the annotation file corresponding to the images you want to work with
annFile =ann_file
# Create a COCO object instance
coco = COCO(annFile)

# Get the IDs of the images
imgIds = coco.getImgIds()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
ds = hub.empty('hub://aismail2/roboflow_pepper_valid',overwrite=True)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/aismail2/roboflow_pepper_valid
hub://aismail2/roboflow_pepper_valid loaded successfully.


In [5]:
class_names=["pepper"]
class_map={"pepper":0}

In [6]:
def filter_data(bbox):
    minw=3
    minh=3
    if ((bbox[2]-bbox[0])<minw or (bbox[3]-bbox[1])<minh):
        return True
    else:
        return False

In [7]:
with ds:
    ds.create_tensor('images', htype='image',sample_compression = 'jpeg')
    ds.create_tensor('boxes', htype='bbox')
    ds.boxes.info.update(coords = {'type': 'pixel', 'mode': 'LTRB'})
    ds.create_tensor('labels', htype='class_label', class_names = class_names)
    ds.create_tensor('masks', htype = 'binary_mask', sample_compression = 'lz4')
    for i in range(len(imgIds)):
        img_info = coco.loadImgs(imgIds[i])[0]
        height=img_info["height"]
        width=img_info["width"]
        img_path=os.path.join(img_folder,img_info['file_name'])
        assert os.path.exists(img_path)
        # Get the IDs of the annotations for this image
        annIds = coco.getAnnIds(imgIds=img_info['id'])
        # Load the annotations
        anns = coco.loadAnns(annIds)
        # Append data to tensors
        boxes=[]
        class_labels=[]
        masks=[]
        skip=False
        for ann in anns:
            if 'segmentation' in ann:
                # Get the segmentation (in the form of a binary mask)
                binary_mask = coco.annToMask(ann)
                nz=np.nonzero(binary_mask)
                ys=nz[0]
                xs=nz[1]
                boxes.append([min(xs),min(ys),max(xs),max(ys)])
                class_labels.append(class_map["pepper"])                                   
                masks.append(binary_mask.astype(np.uint8)*255)
        if (len(boxes))==0:
            print(f"Skipping no label!!")
            continue
        if skip:
            print(f"Skipping Image due to very small image")
            continue
        boxes=np.array(boxes)
        masks=np.array(masks)
        # make count last axis for masks require for vis by the tool
        masks=np.moveaxis(masks, 0, -1)
        class_labels=np.array(class_labels)
        ds.images.append(hub.read(img_path))
        ds.labels.append(class_labels.astype(np.uint32))
        ds.boxes.append(boxes.astype(np.float32))
        ds.masks.append(masks.astype(bool))

In [8]:
len(ds)

27